# Gaussian Path Planning test

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pickle
from swarm_prm.utils.gaussian_prm import *
from swarm_prm.envs.roadmap import Roadmap, Obstacle 
from swarm_prm.envs.instance import Instance

roadmap = Roadmap(100, 100)
safety_radius = 2 
pos = np.array([50, 50])
obstacles  = [
    Obstacle(None, "POLYGON", [(40, 0), (40, 40), (80, 40), (80, 0)]),
    Obstacle(None, "POLYGON", [(40, 100), (40, 60), (80, 60), (80, 100)])
]
num_samples = 200 
for obs in obstacles:
    roadmap.add_obstacle(obs)
# fig, ax = roadmap.visualize()
start_mean = np.random.rand(2) * 100
goal_mean = np.random.rand(2) * 100

# Agent config
sampling_method = "GAUSSIAN_SAMPLING"
#sampling_method = "SWARMPRM_SAMPLING"
hex_radius = 4
TEST_CVAR = False


# Curated test instance
starts = np.array([[10, 10], [10, 90]])
goals = np.array([[90, 90], [90, 10]])

starts_weight = np.array([.8, .2])
goals_weight = np.array([.5, .5])

g_starts = [GaussianGraphNode(start, None, "UNIFORM", radius=10) for start in starts]
g_goals = [GaussianGraphNode(goal, None, "UNIFORM", radius=10) for goal in goals]

instance = Instance(roadmap, g_starts, g_goals, starts_weight, goals_weight)

gaussian_prm = GaussianPRM(instance, num_samples, safety_radius=safety_radius,
                           hex_radius=hex_radius)
gaussian_prm.roadmap_construction()

gaussian_prm.visualize_roadmap("solutions/test_map")
gaussian_prm.visualize_g_nodes("solutions/test_g_nodes")

with open("solutions/gaussian_prm.pkl", "wb") as f:
    pickle.dump(gaussian_prm, f)

In [ ]:
# Agent config

num_agent = 500
agent_radius = 0.1

# Map config
map_type = "corridor"
num_samples = 10

In [ ]:
# import pickle
# from swarm_prm.solvers.macro.teg import TEGGraph

# with open("gaussian_prm.pkl", "rb") as f:
#     gaussian_prm = pickle.load(f)

# num_agent = 120
# agent_radius = 1    
# teg_graph= TEGGraph(gaussian_prm, agent_radius, num_agents=num_agent, max_timestep=500)
# _, flow_dict, _, _, _, _ = teg_graph.get_earliest_timestep()

# from pprint import pprint
# pprint(flow_dict)

# with open("flow_dict.pkl", "wb") as f:
#     pickle.dump(flow_dict, f)

In [ ]:
import pickle
from swarm_prm.solvers.macro.mcf import TEG_MCF

with open("solutions/gaussian_prm.pkl", "rb") as f:
    gaussian_prm = pickle.load(f)

num_agent = 120
agent_radius = 1    
teg_graph= TEG_MCF(gaussian_prm, num_agent, agent_radius, max_timestep=500)
timestep, flow_dict, residual_graph = teg_graph.get_earliest_timestep()

with open("solutions/flow_dict.pkl", "wb") as f:
    pickle.dump(flow_dict, f)

In [ ]:
import pickle

with open("solutions/gaussian_prm.pkl", "rb") as f:
    gaussian_prm = pickle.load(f)

with open("solutions/flow_dict.pkl", "rb") as f:
    flow_dict = pickle.load(f)

macro_sol = gaussian_prm.get_macro_solution(flow_dict)
simple_paths, gaussian_paths = gaussian_prm.get_solution(flow_dict, timestep, num_agent)
with open("solutions/macro_sol.pkl", "wb") as f:
    pickle.dump(macro_sol, f)

In [ ]:
print(len(simple_paths))

In [ ]:
gaussian_prm.visualize_g_nodes()
gaussian_prm.visualize_roadmap()
fig, ax = instance.visualize()

# apf_solver.animate_solution(fig, ax)
for path in simple_paths:
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-*', label='Path', color='blue')
# 

## Gaussian Trajectory

In [ ]:
import pickle
from matplotlib import pyplot as plt
from swarm_prm.solvers.micro import GaussianTrajectorySolver

with open("solutions/gaussian_prm.pkl", "rb") as f:
    gaussian_prm = pickle.load(f)

with open("solutions/macro_sol.pkl", "rb") as f:
    macro_sol = pickle.load(f)

timestep = max(macro_sol.keys())

gt_solver = GaussianTrajectorySolver(gaussian_prm, macro_sol, timestep, num_agent, safety_gap=0.2)
gt_paths = gt_solver.solve()

fig, ax = instance.visualize()

for path in simple_paths:
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-*', label='Path', color='blue')
# 
cmap = plt.get_cmap("tab10")

for i, path in enumerate(gt_paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '--', label='Path', color=cmap(i%10))


In [ ]:
# Add interpolations 

import numpy as np

def add_linear_interpolation_points(path, subdiv=5):
    """
    Insert linearly-interpolated positions between consecutive points.
    
    Parameters
    ----------
    path : array-like of shape (N, 2)
        The original list/array of N 2D points, e.g. [(x1,y1), (x2,y2), ...].
    subdiv : int
        Number of intervals to subdivide each segment. 
        For each original segment, we will add 'subdiv - 1' new interior points 
        (except that we skip duplicates at the shared boundary).

    Returns
    -------
    new_path : np.ndarray of shape (M, 2)
        The new path including the original points and the added interpolation points.
    """
    path = np.asarray(path)
    new_path = []

    # Iterate over pairs of consecutive points
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        
        # Create subdiv+1 points in [start, end] using linspace for each dimension
        xs = np.linspace(start[0], end[0], subdiv + 1)
        ys = np.linspace(start[1], end[1], subdiv + 1)
        
        # Add each intermediate point except the very last one
        # to avoid duplicating the next segment's start
        for j in range(subdiv):
            new_path.append([xs[j], ys[j]])

    # Finally add the very last point of the last segment
    new_path.append(path[-1].tolist())
    
    return np.array(new_path)

interpolated_paths = []
for path in gt_paths:
    new_path = add_linear_interpolation_points(path, 50)
    interpolated_paths.append(new_path)


In [ ]:
def animate_solution(fig, ax, speed, paths):
    """
        Visualize solution trajectory provided instance
    """
    
    agents = []
    def init():
        for agent in agents:
            agent.remove()
        agents.clear()
        return []

    def update(frame):
        idx = frame * speed
        for agent in agents:
            agent.remove()
        agents.clear()
        cmap = plt.get_cmap("tab10")
        locs = [path[idx] for path in paths]
        for i, loc in enumerate(locs):
            agent = ax.add_patch(Circle(loc, radius=agent_radius, color=cmap(i%10)))
            agents.append(agent)
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0])// speed , 
                         init_func=init, blit=True, interval=10)
    anim.save("solutions/apf_solution.gif", writer='pillow', fps=24)

# fig, ax = instance.visualize()
fig, ax = gaussian_prm.visualize_g_nodes()
animate_solution(fig, ax, 1, interpolated_paths)